# Домашняя работа по регуляризации и оптимизации

Ниже приводится корпус данных с двумя метками: 1 и -1. К данным применяется линейная модель классификации:

$f(x, \theta) = x_1 \theta_1 + x_2 \theta_2 + \theta_3.$

Предлагается подобрать параметры $\theta$ минимизируя следующую функцию ошибки:

$\mathcal{L}(\theta) = 0.1 \|\theta\|^2 + \frac{1}{N}\sum\limits_{i=1}^N \max(0, 1 - y_i f(x_i, \theta)).$

Для оптимизации предлагается использовать метод градиентного спуска с 1000 шагами размера $0.1$ из начальной точки $(1, 1, 0)$.

In [1]:
import numpy as np
import yaml

In [30]:
X = np.array([
    [0, 1],
    [1, 1],
    [1, 0],
    [-0.5, 0.5],
    [0, -0.5]
])

y = np.array([1, 1, 1, -1, -1])

theta0 = np.array([1.0, 1.0, 0.0])

lr = 0.1

def f(X, theta):
    theta = np.asarray(theta)
    return (X * theta[:2]).sum(axis=-1) + theta[2]

def loss(X, y, theta):
    theta = np.asarray(theta)
    norm = (theta ** 2).sum()
    deltas = y * f(X, theta)
    return 0.1 * norm + np.mean(np.maximum(0, 1 - deltas))

print("Prediction:", f(X, theta0))
print("Loss:", loss(X, y, theta0))

Prediction: [ 1.   2.   1.   0.  -0.5]
Loss: 0.5


In [73]:
# Якобиан f(X, theta)

def A_f(X):
  d = np.full((5,1), 1)
  return np.column_stack((X, d))

print("Якобиан f(X):\n", A_f(X))

Якобиан f(X):
 [[ 0.   1.   1. ]
 [ 1.   1.   1. ]
 [ 1.   0.   1. ]
 [-0.5  0.5  1. ]
 [ 0.  -0.5  1. ]]


In [74]:
# Якобиан среднего максимумов из 0 и 1-y_i*f(X)

def A_max(y, X, theta):
  res = np.zeros((5, 1))
  for i in range(0, len(y)):
    if 1 > y[i]*f(X, theta)[i]:
      res[i] = -y[i]
  return res.T@A_f(X)/len(y)

In [75]:
# Якобиан loss(X, y, theta)

def A_l(theta):
  theta = np.asarray(theta)
  return 0.2*theta + A_max(y, X, theta)

In [76]:
# Шаг градиентного спуска

def step(theta, lr=0.1):
    A = A_l(theta)[0]
    return theta - lr * A

In [77]:
# 1000 шагов градиентного спуска

theta = theta0
for _ in range(1000):
    theta = step(theta)

print("Конечная theta: ", theta)

Конечная theta:  [ 1.25218494  1.         -0.24781506]


In [78]:
print("Prediction:", f(X, theta))
print("Loss:", loss(X, y, theta))

with open("submission.yaml", "w") as fp:
    yaml.safe_dump({"tasks": [{"task1": {"answer": theta.tolist()}}]}, fp)

Prediction: [ 0.75218494  2.00436988  1.00436988 -0.37390753 -0.74781506]
Loss: 0.48815643632236133
